# Claude Function Calling on Anthropic Tools
- https://github.com/gonsoomoon-ml/anthropic-tools

## 1. 환경 설정
- 아래 링크를 클릭하여 Claude Model 을 사용하기 위한 API Key 설정 하세요.
    - [VS CODE 노트북에서 환경 변수 사용하기](https://github.com/gonsoomoon-ml/Self-Study-Generative-AI/blob/main/environment_tips/README.md)

In [1]:
# install_needed = True
install_needed = False

if install_needed:
    ! pip install -r requirements.txt
    ! pip install python-dotenv


# 2. Tool 정의

In [2]:
from tool_use_package.tool_user import ToolUser
from tool_use_package.tools.base_tool import BaseTool

In [3]:
# Create Tools
class AdditionTool(BaseTool):
    """Tool to add two numbers together."""
    def use_tool(self, a, b):
        return a + b

class SubtractionTool(BaseTool):
    """Tool to subtract one number from another."""
    def use_tool(self, a, b):
        return a - b

# Instantiate Each Tool
addition_tool_name = "perform_addition"
addition_tool_description = "Add two numbers, a and b, together. For example, add_numbers(a=10, b=12) -> 22. Numbers can be any rational number."
addition_tool_parameters = [
    {"name": "a", "type": "float", "description": "The first number to add, such as 5"},
    {"name": "b", "type": "float", "description": "The second number to add, such as 4.6"}
]

subtraction_tool_name = "perform_subtraction"
subtraction_tool_description = "Perform subtraction of one number (b) from another (a) yielding a-b. For example, subtract_numbers(a=8, b=5) -> 3. Numbers can be any rational number."
subtraction_tool_parameters = [
    {"name": "a", "type": "float", "description": "The minuend, such as 5"},
    {"name": "b", "type": "float", "description": "The subtrahend, such as 9"}
]


In [4]:
addition_tool = AdditionTool(addition_tool_name, addition_tool_description, addition_tool_parameters)
subtraction_tool = SubtractionTool(subtraction_tool_name, subtraction_tool_description, subtraction_tool_parameters)


# 3. Tool User 및 메시지 정의

In [5]:


# Instantiate ToolUser by Passing it Tool Instances 
math_tool_user = ToolUser([addition_tool, subtraction_tool])

# Build messages
user_message = {
    "role": "user",
    "content": "Sally has 17 apples. She gives 9 to Jim. Later that day, Peter gives 6 Bananas to Sally. How many pieces of fruit does Sally have at the end of the day?"
}

messages = [user_message]


In [9]:
# print("math_tool_user.client.api_key: ", math_tool_user.client.api_key)
# print("math_tool_user.client.auth_headers: ", math_tool_user.client.auth_headers)
# print("math_tool_user.client.auth_token: ", math_tool_user.client.auth_token)
print("math_tool_user.model: ", math_tool_user.model)

math_tool_user.model:  claude-3-opus-20240229


# 4. 메세지를 Tool 을 사용하여 결과 얻음

In [7]:

# Use Claude With the Provided Tools
math_tool_user.use_tools(messages, execution_mode='automatic')

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

# 5. 수동으로 실행

## 메세지 정의

In [9]:
manual_messages = None
manual_messages = messages
manual_messages

[{'role': 'user',
  'content': 'Sally has 17 apples. She gives 9 to Jim. Later that day, Peter gives 6 Bananas to Sally. How many pieces of fruit does Sally have at the end of the day?'}]

## 메세지를 실행
- 결과 값은 메세지의 tool_inputs 가 나옴

In [10]:
claude_res = math_tool_user.use_tools(manual_messages, execution_mode='manual')
claude_res

{'role': 'tool_inputs',
 'content': "Okay, let's break this down step-by-step:\n* Sally starts with 17 apples\n* She gives 9 apples to Jim\nTo calculate how many apples Sally has left:\n",
 'tool_inputs': [{'tool_name': 'perform_subtraction',
   'tool_arguments': {'a': 17.0, 'b': 9.0}}]}

## 메세지에 tool_inputs 를 추가하여 메세지 업데이트

In [11]:
manual_messages.append(claude_res)
manual_messages

[{'role': 'user',
  'content': 'Sally has 17 apples. She gives 9 to Jim. Later that day, Peter gives 6 Bananas to Sally. How many pieces of fruit does Sally have at the end of the day?'},
 {'role': 'tool_inputs',
  'content': "Okay, let's break this down step-by-step:\n* Sally starts with 17 apples\n* She gives 9 apples to Jim\nTo calculate how many apples Sally has left:\n",
  'tool_inputs': [{'tool_name': 'perform_subtraction',
    'tool_arguments': {'a': 17.0, 'b': 9.0}}]}]

## perform_subtraction 함수를 외부에서 실행한 결과를 메세지로 생성
- 수동으로 진행하기에, 아래는 실제 수동으로 실행한 결과로 가정하고, 메세지를 생성 함.

In [12]:
next_message = {
    "role": "tool_outputs",
    "tool_outputs": [
        {
            "tool_name": "perform_subtraction",
            "tool_result": 8
        }
    ],
    "tool_error": None
}

## 기존 메세지에 perform_subtraction 메세지 추가

In [13]:
manual_messages.append(next_message)
manual_messages

[{'role': 'user',
  'content': 'Sally has 17 apples. She gives 9 to Jim. Later that day, Peter gives 6 Bananas to Sally. How many pieces of fruit does Sally have at the end of the day?'},
 {'role': 'tool_inputs',
  'content': "Okay, let's break this down step-by-step:\n* Sally starts with 17 apples\n* She gives 9 apples to Jim\nTo calculate how many apples Sally has left:\n",
  'tool_inputs': [{'tool_name': 'perform_subtraction',
    'tool_arguments': {'a': 17.0, 'b': 9.0}}]},
 {'role': 'tool_outputs',
  'tool_outputs': [{'tool_name': 'perform_subtraction', 'tool_result': 8}],
  'tool_error': None}]

## 메세지를 실행하여 tool_inputs 를 얻음

In [14]:


claude_res = math_tool_user.use_tools(manual_messages, execution_mode='manual')
claude_res



{'role': 'tool_inputs',
 'content': "\n\nSo after giving 9 apples to Jim, Sally has 8 apples left.\n\n* Later, Peter gives Sally 6 bananas\n\nTo calculate Sally's total fruit at the end of the day, we add her remaining apples and the bananas from Peter:\n\n",
 'tool_inputs': [{'tool_name': 'perform_addition',
   'tool_arguments': {'a': 8.0, 'b': 6.0}}]}

## tool_inputs 추가

In [15]:
manual_messages.append(claude_res)
manual_messages

[{'role': 'user',
  'content': 'Sally has 17 apples. She gives 9 to Jim. Later that day, Peter gives 6 Bananas to Sally. How many pieces of fruit does Sally have at the end of the day?'},
 {'role': 'tool_inputs',
  'content': "Okay, let's break this down step-by-step:\n* Sally starts with 17 apples\n* She gives 9 apples to Jim\nTo calculate how many apples Sally has left:\n",
  'tool_inputs': [{'tool_name': 'perform_subtraction',
    'tool_arguments': {'a': 17.0, 'b': 9.0}}]},
 {'role': 'tool_outputs',
  'tool_outputs': [{'tool_name': 'perform_subtraction', 'tool_result': 8}],
  'tool_error': None},
 {'role': 'tool_inputs',
  'content': "\n\nSo after giving 9 apples to Jim, Sally has 8 apples left.\n\n* Later, Peter gives Sally 6 bananas\n\nTo calculate Sally's total fruit at the end of the day, we add her remaining apples and the bananas from Peter:\n\n",
  'tool_inputs': [{'tool_name': 'perform_addition',
    'tool_arguments': {'a': 8.0, 'b': 6.0}}]}]

## perform_addition 로 실행 결과를 얻었다고 가정하여 메세지 생성

In [16]:

next_message = {
    "role": "tool_outputs",
    "tool_outputs": [
        {
            "tool_name": "perform_addition",
            "tool_result": 14
        }
    ],
    "tool_error": None
}

## tool_outputs 생성

In [17]:
manual_messages.append(next_message)
manual_messages

[{'role': 'user',
  'content': 'Sally has 17 apples. She gives 9 to Jim. Later that day, Peter gives 6 Bananas to Sally. How many pieces of fruit does Sally have at the end of the day?'},
 {'role': 'tool_inputs',
  'content': "Okay, let's break this down step-by-step:\n* Sally starts with 17 apples\n* She gives 9 apples to Jim\nTo calculate how many apples Sally has left:\n",
  'tool_inputs': [{'tool_name': 'perform_subtraction',
    'tool_arguments': {'a': 17.0, 'b': 9.0}}]},
 {'role': 'tool_outputs',
  'tool_outputs': [{'tool_name': 'perform_subtraction', 'tool_result': 8}],
  'tool_error': None},
 {'role': 'tool_inputs',
  'content': "\n\nSo after giving 9 apples to Jim, Sally has 8 apples left.\n\n* Later, Peter gives Sally 6 bananas\n\nTo calculate Sally's total fruit at the end of the day, we add her remaining apples and the bananas from Peter:\n\n",
  'tool_inputs': [{'tool_name': 'perform_addition',
    'tool_arguments': {'a': 8.0, 'b': 6.0}}]},
 {'role': 'tool_outputs',
  'too

## 최종 실행하여 결과 얻음

In [18]:
math_tool_user.use_tools(manual_messages, execution_mode='manual')

{'role': 'assistant',
 'content': '\n\n\n\nTherefore, at the end of the day, Sally has a total of 14 pieces of fruit - 8 apples and 6 bananas.'}